*مجموعه داده:*
---
پیش‌پردازش داده‌ها در حالت کلی الزامی نیست، زیرا این مجموعه داده از قبل آماده‌سازی شده و ویژگی‌های مورد نیاز را شامل می‌شود.
دلایل عدم نیاز به پیش‌پردازش سنگین:

1. ساختار داده‌های آماده:
    * هر نمونه شامل سه ویژگی اصلی است:
        article (متن اصلی مقاله).
        highlights (خلاصه متن).
        id (شناسه یکتا).

    * داده‌ها از نظر قالب و محتوا از قبل تمیز و سازمان‌دهی شده‌اند.

2. عدم وجود داده‌های نامعتبر یا ناقص:
    * هیچ داده نامعتبر (null یا NaN) در مجموعه داده شناسایی نشده است.

3. فرمت‌بندی مناسب:
    * متن و خلاصه‌ها از نظر ساختار زبانی منسجم هستند و نیاز به تصحیح یا تغییر قالب ندارند.

In [ ]:
!pip install -q -U bitsandbytes
!pip install transformers
!pip install peft
!pip install -q datasets
!pip install -qqq trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 9.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.6 MB/s eta 0:00:00:00:01


In [ ]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import warnings
warnings.filterwarnings("ignore")


In [ ]:
huggingface_dataset_name = "cnn_dailymail"

dataset = load_dataset(huggingface_dataset_name, "3.0.0")
dataset


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
def format_instruction(dialogue: str, summary: str):
    return f"""### Instruction:
Summarize the following conversation.

### Input:
{dialogue.strip()}

### Summary:
{summary}
""".strip()

def generate_instruction_dataset(data_point):

    return {
        "article": data_point["article"],
        "highlights": data_point["highlights"],
        "text": format_instruction(data_point["article"],data_point["highlights"])
    }

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(['id'])
    )
    

In [ ]:
dataset["train"] = process_dataset(dataset["train"])
dataset["test"] = process_dataset(dataset["validation"])
dataset["validation"] = process_dataset(dataset["validation"])


train_data = dataset['train'].shuffle(seed=42).select([i for i in range(5000)])

validation_data = dataset['validation'].shuffle(seed=42).select([i for i in range(500)])
test_data = dataset['test'].shuffle(seed=42).select([i for i in range(100)])


train_data,test_data,validation_data


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

(Dataset({
     features: ['article', 'highlights', 'text'],
     num_rows: 5000
 }),
 Dataset({
     features: ['article', 'highlights', 'text'],
     num_rows: 100
 }),
 Dataset({
     features: ['article', 'highlights', 'text'],
     num_rows: 500
 }))

In [ ]:
# Analyze and print one sample
print("Sample data:")
print(train_data[0])

# Find longest and shortest articles and summaries
longest_text = max(train_data, key=lambda x: len(x["article"]))
shortest_text = min(train_data, key=lambda x: len(x["article"]))
longest_summary = max(train_data, key=lambda x: len(x["highlights"]))
shortest_summary = min(train_data, key=lambda x: len(x["highlights"]))

print("\nLongest Article:")
print(longest_text["article"])
print("\nShortest Article:")
print(shortest_text["article"])
print("\nLongest Summary:")
print(longest_summary["highlights"])
print("\nShortest Summary:")
print(shortest_summary["highlights"])


# Calculate the mean length of articles
mean_article_length = sum(len(x["article"]) for x in train_data) / len(train_data)

# Calculate the mean length of summaries
mean_summary_length = sum(len(x["highlights"]) for x in train_data) / len(train_data)

print(f"Mean Article Length: {mean_article_length:.2f} characters")
print(f"Mean Summary Length: {mean_summary_length:.2f} characters")


Sample data:
{'article': "Fifty Shades of Grey is set to feature more sex on screen than the 100 raunchiest films released in 2014 put together, making it the most erotic mainstream movie in a decade. The film, which premieres on Valentine's Day features a dozen sex scenes, which make up 20 minutes of its total 100-minute running time. This follows an interview with actor Jamie Dornan, who plays main man Christian Grey, where he says he 'does not believe it is pornographic or even erotic.' Scroll down for video . 'Not even erotic': Jamie Dornan and\xa0Dakota Johnson star as Christian Grey and Anastasia Steele in the upcoming filmatisation of Fifty Shades of Grey, claimed to be the 'raunchiest' film in a decade . The 'raunchiest of the decade' accolade has been awarded Fifty Shades after website Mr Skin, which ranks films by amount of sex scenes, released the data seen by the Sunday Times. Fifty Shades Of Grey, which stars Dakota Johnson as Anastasia Steele and\xa0Dornan as the BDSM-lov

In [ ]:
from huggingface_hub import login

login("MY TOKEN")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-1B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config, 
    device_map="auto", 
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;">

*بخش اول:*
---

مقایسه نتایج Zero-Shot و Few-Shot
* Zero-Shot

    * ROUGE:
        rouge1: 0.1036، rouge2: 0.0693، rougeL: 0.0950

    * BERTScore:
        Precision: 0.7927، Recall: 0.8946، F1: 0.8406

* Few-Shot

    * ROUGE:
        rouge1: 0.0276، rouge2: 0.0021، rougeL: 0.0191

    * BERTScore:
        Precision: 0.7518، Recall: 0.7957، F1: 0.7731

* تحلیل

    * Zero-Shot بهتر عمل می‌کند:
        در هر دو معیار ROUGE و BERTScore، Zero-Shot عملکرد بهتری دارد.
        
        Recall و F1 بالاتر نشان‌دهنده همخوانی بهتر با معنای متن مرجع است.

    * Few-Shot ضعیف‌تر است:
        
        ROUGE بسیار پایین‌تر است، نشان‌دهنده مشکل در تولید خلاصه‌های مشابه متن مرجع.
        
        Precision پایین‌تر نشان‌دهنده محتوای غیرمرتبط بیشتر است.

---

مفاهیم:
1. چالش‌های ارزیابی مدل‌های زبانی در تولید متن خلاصه‌سازی

    * چالش‌ها:

        ذهنی بودن معیارها: ارزیابی خلاصه‌سازی تا حدی ذهنی است، زیرا یک متن خلاصه می‌تواند به روش‌های مختلف صحیح باشد.

        عدم تطابق معیارها با کیفیت انسانی: معیارهایی مثل ROUGE بیشتر بر شباهت لغوی تکیه دارند و ممکن است خلاصه‌های معنایی درست را کم‌ارزش کنند.
        
        تفاوت سبک نگارشی: خلاصه‌های تولیدی ممکن است سبک متفاوتی نسبت به مرجع داشته باشند، حتی اگر معنایی مشابه ارائه دهند.

2. توضیح معیارهای ROUGE و BERTScore

    * ROUGE:
    
        مزایا: بر اساس همپوشانی n-gram بین خلاصه تولیدشده و مرجع عمل می‌کند و در مقایسه خلاصه‌های لغوی مناسب است.
        معایب: به خلاصه‌هایی که بازنویسی شده یا سبک متفاوتی دارند، امتیاز پایینی می‌دهد.
    
    * BERTScore:
    
        مزایا: به جای شباهت لغوی، از مدل‌های زبان بزرگ برای مقایسه معنایی خلاصه استفاده می‌کند.
        معایب: نیازمند منابع محاسباتی بیشتر بوده و گاهی به خروجی‌های کلی امتیاز بالا می‌دهد.

3. تأثیر کوانتایزر 4 بیتی (Quantization)

    * علت استفاده: کاهش حافظه مورد نیاز و افزایش سرعت در پردازش مدل‌ها.
    
    * تأثیر:
    
        سرعت: پردازش سریع‌تر در محیط‌هایی با منابع محدود.
        دقت: ممکن است باعث افت جزیی در دقت مدل شود، به خصوص در وظایف حساس به جزئیات.

4. انتخاب نمونه‌های ورودی در Few-Shot

    * معیارها:
    
        انتخاب نمونه‌هایی که پوشش‌دهی خوبی از موضوعات و سبک‌های مختلف متن داشته باشند.
    
        انتخاب مثال‌های ساده و واضح برای راهنمایی بهتر مدل.
    
    * تأثیر: نمونه‌های مناسب می‌توانند عملکرد مدل را در خلاصه‌سازی بهبود بخشند.

5. مقایسه نسخه ساده و نسخه Instruct-Tune

    * نسخه ساده:
    
        بدون آموزش اضافی و مستقیم از مدل پیش‌آموزش‌دیده استفاده می‌کند.
    
        مزیت: سریع و ساده.
    
        معایب: عملکرد ضعیف‌تر در وظایف خاص.
    
    * نسخه Instruct-Tune:
    
        با تنظیم مدل روی داده‌های خاص، عملکرد بهتری در وظایف مرتبط مانند خلاصه‌سازی دارد.
    
        مزیت: دقت و کیفیت بالاتر.
    
        معایب: نیاز به منابع بیشتر برای آموزش.

In [ ]:
# Zero-Shot Summarization
def zero_shot_summarization(test_data, model, tokenizer):
    # Select a single sample from the test dataset
    sample = test_data[0]
    input_text = sample["article"]

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to("cuda")  
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=294,
            num_beams=4,
            early_stopping=True
        )

    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\n=== Zero-Shot Summarization ===")
    print(f"Input Article:\n{input_text}\n")
    print(f"Generated Summary:\n{generated_summary}\n")

    return input_text, generated_summary

article, generated_summary = zero_shot_summarization(test_data, model, tokenizer)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



=== Zero-Shot Summarization ===
Input Article:
A £1billion tidal lagoon off the coast of south Wales will form the centrepiece of ambitious renewable energy plans to be unveiled today. George Osborne is set to use his Budget today to announce that the Government is beginning formal talks on funding the project to produce energy from turbines in Swansea Bay, south Wales. The scheme would be over an area of more than four square miles cordoned off by a breakwater, with power generated as the tides pass through turbines when going in and out. Scroll down for video . Construction: The project includes creating a six-mile sea reef, the reintroduction of the native oyster to Swansea Bay and an offshore visitor centre as well as national triathlon and water sports facilities . Below the surface: This graphic shows how the structure would generate energy at high tide, left, and low tide, right, meaning that water will flow through four times daily . Vast: The scheme would be over an area of m

In [ ]:
# Few-Shot Summarization
def few_shot_summarization(train_data, test_data, model, tokenizer):
    # Select three few-shot examples from the training data
    few_shot_examples = train_data.select([0, 1, 2])
    input_text = test_data[0]["article"]


    prompt = "### Few-Shot Examples:\n"
    for example in few_shot_examples:
        prompt += (
            f"Article: {example['article']}\n"
            f"Summary: {example['highlights']}\n\n"
        )

    prompt += f"### Target Article:\n{input_text}\n\n### Predicted Summary:\n"


    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=1024
    ).to("cuda")


    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=150,
            num_beams=4,
            early_stopping=True
        )


    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\n=== Few-Shot Summarization ===")
    print(f"Few-Shot Prompt:\n{prompt}\n")
    print(f"Generated Summary:\n{generated_summary}\n")

    return input_text, generated_summary


article, generated_summary_fewshot = few_shot_summarization(train_data, test_data, model, tokenizer)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



=== Few-Shot Summarization ===
Few-Shot Prompt:
### Few-Shot Examples:
Article: Fifty Shades of Grey is set to feature more sex on screen than the 100 raunchiest films released in 2014 put together, making it the most erotic mainstream movie in a decade. The film, which premieres on Valentine's Day features a dozen sex scenes, which make up 20 minutes of its total 100-minute running time. This follows an interview with actor Jamie Dornan, who plays main man Christian Grey, where he says he 'does not believe it is pornographic or even erotic.' Scroll down for video . 'Not even erotic': Jamie Dornan and Dakota Johnson star as Christian Grey and Anastasia Steele in the upcoming filmatisation of Fifty Shades of Grey, claimed to be the 'raunchiest' film in a decade . The 'raunchiest of the decade' accolade has been awarded Fifty Shades after website Mr Skin, which ranks films by amount of sex scenes, released the data seen by the Sunday Times. Fifty Shades Of Grey, which stars Dakota Johns

In [ ]:
!pip install rouge_score bert_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3a84c7e5169d396d55ea7b29f52c5c49eb66dc28525b75765f5a600f181be137
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer
from bert_score import score

def evaluate_with_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = {"rouge1": [], "rouge2": [], "rougeL": []}
    
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        results["rouge1"].append(scores["rouge1"].fmeasure)
        results["rouge2"].append(scores["rouge2"].fmeasure)
        results["rougeL"].append(scores["rougeL"].fmeasure)
    
    # Calculate average ROUGE scores
    avg_results = {metric: sum(scores) / len(scores) for metric, scores in results.items()}
    return avg_results

def evaluate_with_bertscore(predictions, references, lang="en"):
    P, R, F1 = score(predictions, references, lang=lang)
    return {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

# Evaluate Zero-Shot Summarization
print("\n=== Evaluation for Zero-Shot Summarization ===")
zero_shot_rouge = evaluate_with_rouge([generated_summary], [test_data[0]["highlights"]])
zero_shot_bertscore = evaluate_with_bertscore([generated_summary], [test_data[0]["highlights"]])

print("Zero-Shot ROUGE Scores:", zero_shot_rouge)
print("Zero-Shot BERTScore:", zero_shot_bertscore)

# Evaluate Few-Shot Summarization
print("\n=== Evaluation for Few-Shot Summarization ===")
few_shot_rouge = evaluate_with_rouge([generated_summary_fewshot], [test_data[0]["highlights"]])
few_shot_bertscore = evaluate_with_bertscore([generated_summary_fewshot], [test_data[0]["highlights"]])

print("Few-Shot ROUGE Scores:", few_shot_rouge)
print("Few-Shot BERTScore:", few_shot_bertscore)



=== Evaluation for Zero-Shot Summarization ===


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Zero-Shot ROUGE Scores: {'rouge1': 0.10359712230215828, 'rouge2': 0.06926406926406926, 'rougeL': 0.0949640287769784}
Zero-Shot BERTScore: {'precision': 0.7927143573760986, 'recall': 0.8945730924606323, 'f1': 0.840569257736206}

=== Evaluation for Few-Shot Summarization ===


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Few-Shot ROUGE Scores: {'rouge1': 0.027571580063626724, 'rouge2': 0.0021253985122210418, 'rougeL': 0.019088016967126194}
Few-Shot BERTScore: {'precision': 0.7517890930175781, 'recall': 0.7956898808479309, 'f1': 0.7731167674064636}


<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;">

*بخش دوم:*
---
1. نتایج پس از تنظیم دقیق (Fine-Tuning):

    * ROUGE Scores:
        
        rouge1: 0.2131
        rouge2: 0.1101
        rougeL: 0.1536
    
    * BERTScore:
        
        Precision: 0.8080
        Recall: 0.8784
        F1: 0.8416

2. مقایسه با Zero-Shot و Few-Shot:

    * بهبود در ROUGE:
        
        rouge1 از 0.1036 (Zero-Shot) به 0.2131 افزایش یافته است.
        
        rouge2 نیز از 0.0693 (Zero-Shot) به 0.1101 بهبود یافته است.
        
        این نشان‌دهنده افزایش شباهت ن-گرم‌ها و ساختارهای زبانی میان خلاصه تولیدی و متن مرجع است.
    
    * BERTScore:
        
        F1 در مقایسه با Zero-Shot (0.8406) تقریباً ثابت مانده است (0.8416)، اما همچنان نسبت به Few-Shot (0.7731) بهتر است.
        
        Precision و Recall نیز بهبودهای اندکی نشان می‌دهند.

در نهایت، تنظیم دقیق مدل به طور واضح عملکرد را در خلاصه‌سازی متون بهبود داده است.

---

مفاهیم:

1. سناریوی استفاده از مدل‌های عام (بدون آموزش) و ضرورت تنظیم دقیق:

    * استفاده از مدل‌های بزرگ عمومی:
        
        مناسب برای وظایف عمومی که نیاز به تطبیق خاصی ندارند (مانند ترجمه ساده یا خلاصه‌سازی عمومی).
        
        در محیط‌هایی با منابع محدود یا زمانی که داده کافی برای تنظیم دقیق در دسترس نیست.
    * ضرورت تنظیم دقیق:
        
        در وظایف خاص (مانند خلاصه‌سازی تخصصی یا شناسایی موجودیت‌ها) تنظیم دقیق ضروری است.
        
        باعث می‌شود مدل به داده‌ها و سبک خاص آن وظیفه تطبیق پیدا کند و دقت به‌طور قابل توجهی بهبود یابد.

---
2. تاثیر استفاده از FP16 بر کارایی مدل و نحوه عملکرد:

    * تاثیر:
        
        کاهش مصرف حافظه تا نصف.
        
        افزایش سرعت محاسبات به دلیل استفاده از عملیات‌های 16 بیتی.
    * چالش‌ها:
        
        ممکن است دقت در وظایف حساس (مانند طبقه‌بندی دقیق) کاهش یابد.
        
        نیاز به پشتیبانی سخت‌افزاری (مانند GPUهای جدیدتر).
    * نحوه عملکرد:
        
        در FP16 اعداد با دقت نصف ذخیره می‌شوند و عملیات‌ها سریع‌تر انجام می‌گیرند.
        
        مناسب برای تنظیم مدل‌های بزرگ در محیط‌های محدود.

---
3. مدل‌های زبانی سری SMOLLM2 و معماری آن‌ها:

    * معرفی SMOLLM2:
        
        طراحی شده برای کارایی بالا در دستگاه‌های کم‌منبع.
        
        ترکیبی از مدل‌های سبک‌تر با قابلیت‌های رقابتی.
    * معماری:
        
        استفاده از فشرده‌سازی پارامترها.
        
        پشتیبانی از FP16 و تکنیک‌های کاهش محاسبات.

        عملکرد نزدیک به مدل‌های بزرگ‌تر در بسیاری از وظایف.

---
4. تکنیک‌های آموزش مدل‌های کوچک زبانی:

    * تکنیک‌های موثر:
        
        کوانتایزر: کاهش دقت محاسبات (مانند 4 بیتی) برای بهبود سرعت.
        
        Transfer Learning: استفاده از مدل‌های بزرگ برای آموزش مدل‌های کوچک‌تر.
        
        فشرده‌سازی: کاهش تعداد پارامترها بدون افت دقت زیاد.
    * در SMOLLM2:
        
        تمرکز بر تعادل بین کاهش اندازه و حفظ دقت.
        
        استفاده از تنظیم دقیق با داده‌های خاص وظیفه.

---
5. مقایسه نسخه ساده و نسخه Instruct-Tune:

    * نسخه ساده:
        
        برای وظایف عمومی مناسب است.
        
        سریع‌تر و نیازمند منابع کمتر.
    * نسخه Instruct-Tune:
        
        برای وظایف تخصصی و خاص، تنظیم شده است.
        
        دقت و کیفیت بالاتری دارد اما نیاز به آموزش اضافی دارد.





In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama3_finetuned",       # Directory to save the model
    evaluation_strategy="steps",          # Evaluate at every X steps
    save_strategy="steps",                # Save checkpoints
    logging_dir="./logs",                 # Directory for logs
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=4,        # Batch size per GPU
    per_device_eval_batch_size=4,         # Batch size for evaluation
    num_train_epochs=3,                   # Number of epochs
    logging_steps=500,                    # Log every 500 steps
    save_steps=1000,                      # Save model every 1000 steps
    eval_steps=500,                       # Evaluate model every 500 steps
    warmup_steps=500,                     # Warmup steps
    fp16=True,                            # Use 16-bit precision
    report_to="tensorboard",              # Log to TensorBoard
    load_best_model_at_end=True,          # Load best model after training
)



In [ ]:
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,2.648200,2.524643
1000,2.513100,2.498239
1500,2.485400,2.486017
2000,2.467900,2.479134
2500,2.485600,2.473739
3000,2.458300,2.472174
3500,2.459100,2.470109


Could not locate the best model at ./llama3_finetuned/checkpoint-3500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=3750, training_loss=2.49962412109375, metrics={'train_runtime': 8582.5291, 'train_samples_per_second': 1.748, 'train_steps_per_second': 0.437, 'total_flos': 8.792210827876762e+16, 'train_loss': 2.49962412109375, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
trainer.save_model("./llama3_finetuned")
tokenizer.save_pretrained("./llama3_finetuned")

print("Fine-tuned model saved!")


Fine-tuned model saved!


In [ ]:
!zip -r "./working_directory_1.zip" "/kaggle/working/llama3_finetuned"


updating: kaggle/working/llama3_finetuned/ (stored 0%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/ (stored 0%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/tokenizer.json (deflated 85%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/scheduler.pt (deflated 56%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/optimizer.pt (deflated 9%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/README.md (deflated 66%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/tokenizer_config.json (deflated 94%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/trainer_state.json (deflated 72%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/special_tokens_map.json (deflated 63%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/adapter_model.safetensors (deflated 8%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000/adapter_config.json (deflated 54%)
updating: kaggle/working/llama3_finetuned/checkpoint-3000

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = "/kaggle/input/llama3.1_finetuned_cnndailymail/pytorch/default/1/kaggle/working/llama3_finetuned/checkpoint-3750"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

print("Fine-tuned model and tokenizer loaded!")


# # Save the fine-tuned model
# trainer.save_model("./llama3_finetuned")
# tokenizer.save_pretrained("./llama3_finetuned")

# print("Fine-tuned model saved!")


Fine-tuned model and tokenizer loaded!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


def generate_summary_finetuned(model, tokenizer, input_text, max_new_tokens=150):
    """
    Generates a summary for the input text using the fine-tuned model.

    Args:
        model (transformers.PreTrainedModel): The fine-tuned model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer for the model.
        input_text (str): The text to summarize.
        max_new_tokens (int): Maximum number of tokens to generate.

    Returns:
        str: The generated summary.
    """
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=294
    ).to("cuda")

    # Generate the summary
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True
        )

    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_summary


In [ ]:
test_predictions = [
    generate_summary_finetuned(model, tokenizer, article["article"])
    for article in test_data
]

test_references = [article["highlights"] for article in test_data]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [ ]:
print("\n=== Llama3.2 Summarization ===")
print(f"Reference Summary:\n{test_references[0]}\n")
print(f"Generated Summary:\n{test_predictions[0]}\n")



=== Llama3.2 Summarization ===
Reference Summary:
Government to begin formal talks on funding Swansea Bay energy project .
Scheme would be over area of four sq miles cordoned off by a breakwater .
Power generated as tides pass through turbines when going in and out .

Generated Summary:
A £1billion tidal lagoon off the coast of south Wales will form the centrepiece of ambitious renewable energy plans to be unveiled today. George Osborne is set to use his Budget today to announce that the Government is beginning formal talks on funding the project to produce energy from turbines in Swansea Bay, south Wales. The scheme would be over an area of more than four square miles cordoned off by a breakwater, with power generated as the tides pass through turbines when going in and out. Scroll down for video. Construction: The project includes creating a six-mile sea reef, the reintroduction of the native oyster to Swansea Bay and an offshore visitor centre as well as national triathlon and wate

In [ ]:
from rouge_score import rouge_scorer
from bert_score import score

# Function to evaluate with ROUGE
def evaluate_with_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = {"rouge1": [], "rouge2": [], "rougeL": []}
    
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        results["rouge1"].append(scores["rouge1"].fmeasure)
        results["rouge2"].append(scores["rouge2"].fmeasure)
        results["rougeL"].append(scores["rougeL"].fmeasure)
    
    # Calculate average ROUGE scores
    avg_results = {metric: sum(scores) / len(scores) for metric, scores in results.items()}
    return avg_results

# Function to evaluate with BERTScore
def evaluate_with_bertscore(predictions, references, lang="en"):
    P, R, F1 = score(predictions, references, lang=lang)
    return {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

# Evaluate ROUGE
rouge_results = evaluate_with_rouge(test_predictions, test_references)
print("ROUGE Scores:", rouge_results)

# Evaluate BERTScore
bert_results = evaluate_with_bertscore(test_predictions, test_references, lang="en")
print("BERTScore:", bert_results)


ROUGE Scores: {'rouge1': 0.21308312878557265, 'rouge2': 0.11005611674701608, 'rougeL': 0.15359150253258858}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': 0.8079621195793152, 'recall': 0.8783740997314453, 'f1': 0.8415759205818176}


<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px;">

*بخش سوم:*
---
1. عملکرد مدل هرس‌شده و مقایسه با مدل تنظیم‌شده:

    * نتایج مدل هرس‌شده (Pruned Model):
        
        ROUGE:
            
            rouge1: 0.1675
            rouge2: 0.0867
            rougeL: 0.1225
        
        BERTScore:
            
            Precision: 0.7891
            Recall: 0.8763
            F1: 0.8302

    * نتایج مدل تنظیم دقیق‌شده پس از هرس (Fine-Tuned Pruned Model):
        
        ROUGE:
            
            rouge1: 0.2129
            rouge2: 0.1102
            rougeL: 0.1529
        
        BERTScore:
            
            Precision: 0.8065
            Recall: 0.8780
            F1: 0.8406

2. تحلیل تاثیر هرس بر عملکرد مدل:

    * تاثیر منفی هرس:
        
        کاهش دقت اولیه در مدل هرس‌شده، به ویژه در مقادیر ROUGE (rouge1 و rouge2) مشاهده می‌شود.
        
        BERTScore نیز افت کمی در F1 و Precision نشان می‌دهد.
        
        این افت ناشی از حذف 30 درصد از وزن‌های مدل است که برخی از آنها اطلاعات مهم را شامل می‌شده‌اند.

3. تاثیر تنظیم دقیق پس از هرس:

    * تنظیم دقیق مدل هرس‌شده باعث بهبود قابل توجه در تمامی متریک‌ها شده است:
        
        ROUGE بهبود یافته و به مقادیر نزدیک به مدل اصلی تنظیم‌شده بازگشته است.
        
        BERTScore نیز بهبود یافته و عملکرد مدل را به سطح اولیه نزدیک کرده است.
        
        تنظیم دقیق باعث می‌شود مدل مجدداً اطلاعات ضروری را با توجه به داده‌های خاص وظیفه یاد بگیرد و اثرات منفی هرس را کاهش دهد.


    مقادیر ROUGE و BERTScore مدل هرس‌شده پس از تنظیم دقیق، نزدیک به مدل تنظیم‌شده اولیه بدون هرس است.
    این نشان می‌دهد که تنظیم دقیق می‌تواند اثرات منفی هرس (مانند حذف اطلاعات مهم) را جبران کند.

---

مفاهیم:

1. تکنیک‌های مختلف هرس مدل‌های زبانی:

    
    * هرس بدون ساختار (Unstructured Pruning): حذف وزن‌های غیرضروری بدون توجه به ساختار مدل.
        
        مزایا: کاهش مصرف حافظه و افزایش sparsity.
        
        معایب: پیچیدگی سخت‌افزاری و نیاز به تنظیم دقیق بیشتر.
    
    * هرس ساختاری (Structured Pruning): حذف کامل بخش‌هایی از مدل (مانند لایه‌ها یا نودها).
        
        مزایا: سادگی در اجرا و افزایش سرعت.
        
        معایب: کاهش دقت بیشتر.
    
    * هرس ترکیبی (Global Sparsity): ترکیب هرس محلی و سراسری.
        
        مزایا: انعطاف‌پذیری بیشتر.
        
        معایب: پیچیدگی بالای تنظیم.

---
2. مقایسه با روش‌های دیگر فشرده‌سازی:

    
    * Quantization: کاهش دقت اعداد (مثل FP16 یا 4 بیتی).
        
        مزایا: کاهش حافظه و سرعت بالا.
        
        معایب: افت بیشتر در وظایف حساس.
    
    * Distillation: انتقال دانش از مدل بزرگ‌تر به مدل کوچک‌تر.
        
        مزایا: کیفیت مناسب در مدل کوچک‌تر.
    
    * Low-rank Factorization: کاهش پارامترها با تجزیه ماتریسی.
        
        مزایا: حفظ تعادل بین فشرده‌سازی و دقت.

---
3. اثر هرس بر عملکرد:

    
    * کاهش حافظه و هزینه محاسبات: به دلیل کاهش وزن‌ها و پارامترها.
    
    * افت دقت اولیه: ناشی از حذف اطلاعات مهم.

---
4. تنظیم دقیق پس از هرس:

    
    * با بازآموزی مدل، افت عملکرد جبران شده و دقت مدل بهبود می‌یابد.
        
    * این مرحله ضروری است تا مدل بر روی داده‌های وظیفه خاص تطبیق یابد.



In [ ]:
import torch.nn.utils.prune as prune

def apply_pruning(model, amount=0.3):
    for name, module in model.named_modules():
        if hasattr(module, "weight") and isinstance(module, (torch.nn.Linear, torch.nn.Conv2d)):
            prune.l1_unstructured(module, name="weight", amount=amount)
            prune.remove(module, "weight")
    return model


total_params = sum(p.numel() for p in model.parameters())
model = apply_pruning(model)

# Calculate sparsity
def calculate_sparsity(model):
    total_params = sum(p.numel() for p in model.parameters())
    zero_params = sum(torch.sum(p == 0).item() for p in model.parameters())
    sparsity = zero_params / total_params
    return sparsity


sparsity = calculate_sparsity(model)
print(f"Model Sparsity after Pruning: {sparsity * 100:.2f}%")


Model Sparsity after Pruning: 30.00%


In [ ]:
import torch.nn as nn

def check_sparsity(model):
    """
    Check the sparsity of the pruned model.
    Prints the sparsity for each layer and overall sparsity.
    """
    total_params = 0
    zero_params = 0

    print(f"{'Layer':<40} {'Sparsity (%)':<15} {'Zero Weights':<15} {'Total Weights':<15}")
    print("-" * 80)
    
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            # Get the total number of weights and the number of zero weights
            total_weights = module.weight.numel()
            zero_weights = (module.weight == 0).sum().item()

            # Compute sparsity for this layer
            sparsity = 100.0 * zero_weights / total_weights
            print(f"{name:<40} {sparsity:<15.2f} {zero_weights:<15} {total_weights:<15}")
            
            # Update overall counts
            total_params += total_weights
            zero_params += zero_weights
    
    # Compute overall sparsity
    overall_sparsity = 100.0 * zero_params / total_params
    print("-" * 80)
    print(f"{'Overall Sparsity':<40} {overall_sparsity:<15.2f} {zero_params:<15} {total_params:<15}")
    return overall_sparsity

check_sparsity(model)


Layer                                    Sparsity (%)    Zero Weights    Total Weights  
--------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj.base_layer 30.00           1258291         4194304        
model.layers.0.self_attn.q_proj.lora_A.default 30.00           9830            32768          
model.layers.0.self_attn.q_proj.lora_B.default 30.00           9830            32768          
model.layers.0.self_attn.k_proj.base_layer 30.00           314573          1048576        
model.layers.0.self_attn.k_proj.lora_A.default 30.00           9830            32768          
model.layers.0.self_attn.k_proj.lora_B.default 30.00           2458            8192           
model.layers.0.self_attn.v_proj.base_layer 30.00           314573          1048576        
model.layers.0.self_attn.v_proj.lora_A.default 30.00           9830            32768          
model.layers.0.self_attn.v_proj.lora_B.default 30.00           2458            819

29.999998676517134

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pruned_model = pruned_model.to(device)

def generate_summary_finetuned(model, tokenizer, input_text, max_new_tokens=294):
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=294
    ).to(device) 
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True
        )

    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_summary

pruned_predictions = [
    generate_summary_finetuned(pruned_model, tokenizer, article["article"])
    for article in test_data
]

# Evaluate ROUGE for pruned model
pruned_rouge_results = evaluate_with_rouge(pruned_predictions, test_references)
print("Pruned Model ROUGE Scores:", pruned_rouge_results)

# Evaluate BERTScore for pruned model
pruned_bert_results = evaluate_with_bertscore(pruned_predictions, test_references, lang="en")
print("Pruned Model BERTScore:", pruned_bert_results)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Pruned Model ROUGE Scores: {'rouge1': 0.16751959284934764, 'rouge2': 0.0867393795313324, 'rougeL': 0.12249557089842027}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pruned Model BERTScore: {'precision': 0.7890657782554626, 'recall': 0.8762536644935608, 'f1': 0.8301979899406433}


In [ ]:
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments


torch.cuda.empty_cache()
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./llama3_finetuned",       # Directory to save the model
    evaluation_strategy="steps",          # Evaluate at every X steps
    save_strategy="steps",                # Save checkpoints
    logging_dir="./logs",                 # Directory for logs
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=2,        # Batch size per GPU
    per_device_eval_batch_size=2,         # Batch size for evaluation
    num_train_epochs=3,                   # Number of epochs
    logging_steps=500,                    # Log every 500 steps
    save_steps=1000,                      # Save model every 1000 steps
    eval_steps=500,                       # Evaluate model every 500 steps
    warmup_steps=500,                     # Warmup steps
    fp16=True,                            # Use 16-bit precision
    report_to="tensorboard",              # Log to TensorBoard
    load_best_model_at_end=True,          # Load best model after training
)

model.config.use_cache = False  # Disable caching

pruned_trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_args,
)

# Fine-tune the pruned model
pruned_trainer.train()


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,2.867100,2.617118
1000,2.584400,2.572099
1500,2.575600,2.557248
2000,2.544300,2.546072
2500,2.536000,2.536603
3000,2.510100,2.530398
3500,2.509900,2.525460
4000,2.497700,2.522212
4500,2.530000,2.519343
5000,2.512900,2.515479


TrainOutput(global_step=7500, training_loss=2.5419594401041667, metrics={'train_runtime': 16387.8792, 'train_samples_per_second': 0.915, 'train_steps_per_second': 0.458, 'total_flos': 8.21527015593984e+16, 'train_loss': 2.5419594401041667, 'epoch': 3.0})

In [ ]:
# Generate summaries using the fine-tuned pruned model
fine_tuned_pruned_predictions = [
    generate_summary_finetuned(model, tokenizer, article["article"])
    for article in test_data
]

# Evaluate ROUGE for fine-tuned pruned model
fine_tuned_pruned_rouge_results = evaluate_with_rouge(fine_tuned_pruned_predictions, test_references)
print("Fine-Tuned Pruned Model ROUGE Scores:", fine_tuned_pruned_rouge_results)

# Evaluate BERTScore for fine-tuned pruned model
fine_tuned_pruned_bert_results = evaluate_with_bertscore(fine_tuned_pruned_predictions, test_references, lang="en")
print("Fine-Tuned Pruned Model BERTScore:", fine_tuned_pruned_bert_results)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

NameError: name 'test_references' is not defined

In [ ]:
# Extract references (gold summaries)
test_references = [article["highlights"] for article in test_data]

# Evaluate ROUGE for fine-tuned pruned model
fine_tuned_pruned_rouge_results = evaluate_with_rouge(fine_tuned_pruned_predictions, test_references)
print("Fine-Tuned Pruned Model ROUGE Scores:", fine_tuned_pruned_rouge_results)

# Evaluate BERTScore for fine-tuned pruned model
fine_tuned_pruned_bert_results = evaluate_with_bertscore(fine_tuned_pruned_predictions, test_references, lang="en")
print("Fine-Tuned Pruned Model BERTScore:", fine_tuned_pruned_bert_results)


Fine-Tuned Pruned Model ROUGE Scores: {'rouge1': 0.21287360187418508, 'rouge2': 0.11021003969683907, 'rougeL': 0.152882660443788}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-Tuned Pruned Model BERTScore: {'precision': 0.806499719619751, 'recall': 0.8780384063720703, 'f1': 0.84063321352005}
